# Analyze past Allegheny County primaries

In [1]:
import pandas as pd
from PyPDF2 import PdfReader
import re

In [2]:
OFF_YEARS = ['11', '13', '15', '17', '19']
ON_YEARS = ['10', '12', '14', '16', '18']
YEARS = ['10', '11', '12', '13', '14', '15', '16', '17', '18', '19']

In [3]:
### Calculate relevant statistics, append year suffix to columns
def calculate_stats(df, year):
    df = df.groupby('precinct').agg({
        'd_reg': 'sum',
        'd_cast': 'sum'
    }).reset_index()

    df['turnout'] = df.d_cast / df.d_reg
    df['ballots_pct_of_total'] = df.d_cast / df.d_cast.sum()
    df['reg_pct_of_total'] = df.d_reg / df.d_reg.sum()

    df.columns = [f'{i}_{year}' if i not in ['precinct'] else i for i in df.columns]
    df = df.set_index('precinct')
    return df

### Remove ward and district and ward markings, but leave ward for Pittsburgh
def format_precinct_name(precinct):
    precinct = precinct.replace(' WD', ' WARD')
    precinct = precinct.replace(' WRD', ' WARD')
    precinct = re.sub(r' DIST.*', '', precinct)

    if precinct.startswith('PITTSBURGH'):
        return precinct
    else:
        return re.sub(r' WARD.*', '', precinct)

In [4]:
def _parse_pdf_precinct_stats(index, precincts, precinct, text, d_reg_regex, d_cast_regex):
    precinct_text_index = text.index(precinct)

    if index == len(precincts) - 1:
        next_precinct_text_index = len(text)
    else:
        next_precinct_text_index = text.index(precincts[index + 1])

    precinct_text = text[precinct_text_index:next_precinct_text_index]
    d_reg = re.findall(d_reg_regex, precinct_text)[0][0]
    d_cast = re.findall(d_cast_regex, precinct_text)[0][0]

    return { 'precinct': precinct, 'd_reg': d_reg, 'd_cast': d_cast }

REGEX_PRECINCT = "STATISTICS\n(.*)\n                                                      VOTES  PERCENT"
REGEX_D_CAST_13 = "BALLOTS CAST - DEMOCRATIC .  .  .  .  .  .\s+(\d+)\s+(\d+\.\d+)?\n?\s+(?:VOTER TURNOUT - TOTAL|BALLOTS CAST - NONPARTISAN|\*+ \(Republican\) \*+)"
REGEX_D_REG_13 = "REGISTERED VOTERS - DEMOCRATIC  .  .  .  .\s+(\d+)\s+(\d+\.\d+)?\n?\s+(?:BALLOTS CAST - TOTAL|REGISTERED VOTERS - NONPARTISAN)"
REGEX_D_CAST_15 = "BALLOTS CAST - DEMOCRATIC .  .  .  .  .  .\s+(\d+)\s+(.*)(\n)?\s+BALLOTS CAST - REPUBLICAN"
REGEX_D_REG_15 = "REGISTERED VOTERS - DEMOCRATIC  .  .  .  .\s+(\d+)   (.*)(\n)?           REGISTERED VOTERS - REPUBLICAN"

def parse_pdf_results(filename):
    reader = PdfReader(filename)
    text = ""

    for page in reader.pages:
        text += page.extract_text() + "\n"

    text = re.sub("Page \d+\n", "", text)
    precincts = re.findall(REGEX_PRECINCT, text)

    d_cast_regex = REGEX_D_CAST_15 if '2015' in filename else REGEX_D_CAST_13
    d_reg_regex = REGEX_D_REG_15 if '2015' in filename else REGEX_D_REG_13

    precincts_with_figures = [
        _parse_pdf_precinct_stats(index, precincts, precinct, text, d_reg_regex, d_cast_regex)
        for index, precinct in enumerate(precincts)
    ]

    df = pd.DataFrame.from_dict(precincts_with_figures)
    df['precinct'] = df.precinct.str.replace(r'^\d+ ', '', regex=True)
    df['d_cast'] = df.d_cast.astype(int)
    df['d_reg'] = df.d_reg.astype(int)

    return df

### Import voting data

##### Import P10

In [5]:
p10 = pd.DataFrame(
    [['ALEPPO', 559910, 159502, 159502/559910, 1, 1]],
    columns=['precinct', 'd_reg_10', 'd_cast_10', 'turnout_10', 'ballots_pct_of_total_10', 'reg_pct_of_total_10'])
p10 = p10.set_index('precinct')

p10.head(1)

,d_reg_10,d_cast_10,turnout_10,ballots_pct_of_total_10,reg_pct_of_total_10
precinct,,,,,
ALEPPO,559910,159502,0.284871,1,1


##### Import P11

In [6]:
p11 = parse_pdf_results('input/primary_2011.pdf')

p11['precinct'] = p11.precinct.apply(format_precinct_name)
p11['precinct'] = p11.precinct.replace({ 'SEWICKLEY HILL': 'SEWICKLEY HILLS' })
p11 = calculate_stats(p11, '11')
p11.head(1)

,d_reg_11,d_cast_11,turnout_11,ballots_pct_of_total_11,reg_pct_of_total_11
precinct,,,,,
ALEPPO,522,101,0.193487,0.000786,0.000959


##### Import P12

In [7]:
p12 = pd.DataFrame(
    [['ALEPPO', 541508, 108174, 108174/541508, 1, 1]],
    columns=['precinct', 'd_reg_12', 'd_cast_12', 'turnout_12', 'ballots_pct_of_total_12', 'reg_pct_of_total_12'])
p12 = p12.set_index('precinct')

p12.head(1)

,d_reg_12,d_cast_12,turnout_12,ballots_pct_of_total_12,reg_pct_of_total_12
precinct,,,,,
ALEPPO,541508,108174,0.199764,1,1


##### Import P13

In [8]:
p13 = parse_pdf_results('input/primary_2013.pdf')

p13['precinct'] = p13.precinct.apply(format_precinct_name)
p13 = calculate_stats(p13, '13')
p13.head(1)

,d_reg_13,d_cast_13,turnout_13,ballots_pct_of_total_13,reg_pct_of_total_13
precinct,,,,,
ALEPPO,506,60,0.118577,0.000488,0.000954


##### Import P14

In [9]:
p14 = pd.DataFrame(
    [['ALEPPO', 527745, 107994, 107994/527745, 1, 1]],
    columns=['precinct', 'd_reg_14', 'd_cast_14', 'turnout_14', 'ballots_pct_of_total_14', 'reg_pct_of_total_14'])
p14 = p14.set_index('precinct')

p14.head(1)

,d_reg_14,d_cast_14,turnout_14,ballots_pct_of_total_14,reg_pct_of_total_14
precinct,,,,,
ALEPPO,527745,107994,0.204633,1,1


##### Import P15

In [10]:
p15 = parse_pdf_results('input/primary_2015.pdf')

p15['precinct'] = p15.precinct.apply(format_precinct_name)
p15 = calculate_stats(p15, '15')
p15.head(1)

,d_reg_15,d_cast_15,turnout_15,ballots_pct_of_total_15,reg_pct_of_total_15
precinct,,,,,
ALEPPO,472,74,0.15678,0.00065,0.00093


##### Import P16

In [11]:
p16 = pd.DataFrame(
    [['ALEPPO', 520055, 233606, 233606/520055, 1, 1]],
    columns=['precinct', 'd_reg_16', 'd_cast_16', 'turnout_16', 'ballots_pct_of_total_16', 'reg_pct_of_total_16'])
p16 = p16.set_index('precinct')

p16.head(1)

,d_reg_16,d_cast_16,turnout_16,ballots_pct_of_total_16,reg_pct_of_total_16
precinct,,,,,
ALEPPO,520055,233606,0.449195,1,1


##### Import P17

In [12]:
p17 = pd.read_csv('input/primary_2017.csv')
p17 = p17[['Precinct Name', 'Democratic Reg Voters', 'Democratic Ballots Cast']]
p17.columns = ['precinct', 'd_reg', 'd_cast']

p17['precinct'] = p17.precinct.apply(format_precinct_name)
p17 = calculate_stats(p17, '17')
p17.head(1)

,d_reg_17,d_cast_17,turnout_17,ballots_pct_of_total_17,reg_pct_of_total_17
precinct,,,,,
ALEPPO,520,123,0.236538,0.001016,0.000967


##### Import P18

In [13]:
p18 = pd.read_csv('input/primary_2018.csv')
p18 = p18[['Precinct Name', 'Democratic Reg Voters', 'Democratic Ballots Cast']]
p18.columns = ['precinct', 'd_reg', 'd_cast']

p18['precinct'] = p18.precinct.apply(format_precinct_name)
p18 = calculate_stats(p18, '18')
p18.head(1)

,d_reg_18,d_cast_18,turnout_18,ballots_pct_of_total_18,reg_pct_of_total_18
precinct,,,,,
ALEPPO,509,133,0.261297,0.001084,0.000946


##### Import P19

In [14]:
p19 = pd.read_csv('input/primary_2019.csv')
p19 = p19[['Precinct Name', 'Democratic Reg Voters', 'Democratic Ballots Cast']]
p19.columns = ['precinct', 'd_reg', 'd_cast']

p19['d_cast'] = p19.d_cast.str.replace(r'.\s', '', regex=True).astype(int)

p19['precinct'] = p19.precinct.apply(format_precinct_name)
p19 = calculate_stats(p19, '19')
p19.head(1)

,d_reg_19,d_cast_19,turnout_19,ballots_pct_of_total_19,reg_pct_of_total_19
precinct,,,,,
ALEPPO,527,98,0.185958,0.00083,0.000955


### Merge primaries; adjust precinct names

In [15]:
primaries = pd.concat([p10, p11, p12, p13, p14, p15, p16, p17, p18, p19], join='outer', axis=1)
primaries = primaries.reset_index()

In [16]:
primaries['precinct'] = primaries.precinct.replace({
    'CASL SHANNON': 'CASTLE SHANNON',
    "OHARA": "O'HARA",
    'ROSSLYN FARM': 'ROSSLYN FARMS',
    'SPRINGDAL BR': 'SPRINGDALE BR',
    'SPRINGDALE TWP': 'SPRINGDALE TP',
    'UP ST CLAIR': 'UPPER ST. CLAIR',
})

primaries['precinct'] = primaries.precinct.str.replace(r'^E ', 'EAST ', regex=True)
primaries['precinct'] = primaries.precinct.str.replace(r'^MT ', 'MOUNT ', regex=True)
primaries['precinct'] = primaries.precinct.str.replace(r'^N ', 'NORTH ', regex=True)
primaries['precinct'] = primaries.precinct.str.replace(r'^S ', 'SOUTH ', regex=True)
primaries['precinct'] = primaries.precinct.str.replace(r'^W ', 'WEST ', regex=True)
primaries['precinct'] = primaries.precinct.str.replace(' HL', ' HILLS')
primaries['precinct'] = primaries.precinct.str.replace(' HTS', ' HEIGHTS')
primaries['precinct'] = primaries.precinct.str.replace(' HT', ' HEIGHTS')
primaries['precinct'] = primaries.precinct.str.replace(' PK', ' PARK')
primaries['precinct'] = primaries.precinct.str.replace(' VILL', ' VILLAGE')

In [17]:
primaries.head(5)

,precinct,d_reg_10,d_cast_10,turnout_10,ballots_pct_of_total_10,reg_pct_of_total_10,d_reg_11,d_cast_11,turnout_11,ballots_pct_of_total_11,...,d_reg_18,d_cast_18,turnout_18,ballots_pct_of_total_18,reg_pct_of_total_18,d_reg_19,d_cast_19,turnout_19,ballots_pct_of_total_19,reg_pct_of_total_19
0,ALEPPO,559910.0,159502.0,0.284871,1.0,1.0,522,101,0.193487,0.000786,...,509,133,0.261297,0.001084,0.000946,527,98,0.185958,0.000830,0.000955
1,ASPINWALL,NaN,NaN,NaN,NaN,NaN,1092,190,0.173993,0.001479,...,1191,514,0.431570,0.004189,0.002213,1225,363,0.296327,0.003073,0.002219
2,AVALON,NaN,NaN,NaN,NaN,NaN,2112,431,0.204072,0.003355,...,2086,458,0.219559,0.003733,0.003876,2126,358,0.168391,0.003031,0.003852
3,BALDWIN BR,NaN,NaN,NaN,NaN,NaN,9024,2207,0.244570,0.017178,...,8676,1609,0.185454,0.013113,0.016120,8748,1844,0.210791,0.015611,0.015849
4,BALDWIN TP,NaN,NaN,NaN,NaN,NaN,874,190,0.217391,0.001479,...,806,162,0.200993,0.001320,0.001498,836,197,0.235646,0.001668,0.001515


### Analyze off-year primaries

##### Average % of total ballots cast

In [18]:
primaries['ballots_pct_of_total_avg'] = primaries[[
    c for c in primaries.columns if c.startswith('ballots_pct_of_total') and c[-2:] in OFF_YEARS
]].mean(axis=1)

In [19]:
primaries.sort_values(by='ballots_pct_of_total_avg', ascending=False).head(5)

,precinct,d_reg_10,d_cast_10,turnout_10,ballots_pct_of_total_10,reg_pct_of_total_10,d_reg_11,d_cast_11,turnout_11,ballots_pct_of_total_11,...,d_cast_18,turnout_18,ballots_pct_of_total_18,reg_pct_of_total_18,d_reg_19,d_cast_19,turnout_19,ballots_pct_of_total_19,reg_pct_of_total_19,ballots_pct_of_total_avg
93,PITTSBURGH WARD 14,NaN,NaN,NaN,NaN,NaN,20477,6227,0.304097,0.048468,...,6282,0.278915,0.051197,0.041848,23833,6433,0.269920,0.054462,0.043179,0.054936
84,PENN HILLS,NaN,NaN,NaN,NaN,NaN,21593,4405,0.204001,0.034286,...,4656,0.216861,0.037946,0.039892,21987,4954,0.225315,0.041941,0.039835,0.033318
98,PITTSBURGH WARD 19,NaN,NaN,NaN,NaN,NaN,13364,3133,0.234436,0.024386,...,2849,0.219323,0.023219,0.024136,13222,2626,0.198608,0.022232,0.023955,0.028750
73,MOUNT LEBANON,NaN,NaN,NaN,NaN,NaN,12394,3046,0.245764,0.023709,...,3945,0.292287,0.032151,0.025078,14028,3179,0.226618,0.026914,0.025415,0.025922
153,WEST MIFFLIN,NaN,NaN,NaN,NaN,NaN,10451,3415,0.326763,0.026581,...,2025,0.216717,0.016503,0.017361,9372,2555,0.272621,0.021631,0.016980,0.023907


In [20]:
primaries['ballots_pct_of_total_avg_dw'] = 100 * primaries.ballots_pct_of_total_avg
primaries[['precinct', 'ballots_pct_of_total_avg_dw']].to_csv('output/ballots_pct_of_total_avg_dw.csv', index=False)

In [21]:
count = 26
ballots_pct = primaries.sort_values(by='ballots_pct_of_total_avg', ascending=False).head(count).ballots_pct_of_total_avg.sum()
reg_pct = primaries.sort_values(by='ballots_pct_of_total_avg', ascending=False).head(count).reg_pct_of_total_19.sum()

print('It takes **%i** munis/PGH wards to reach:' % count)
print('-> This average pct of the off-year primary vote: %s\n' % ballots_pct)
print('-> With this pct of registered Democrats: %s\n' % reg_pct)

It takes **26** munis/PGH wards to reach:
-> This average pct of the off-year primary vote: 0.5009697440601389

-> With this pct of registered Democrats: 0.4703481262059407



##### East End PGH vs. non-East-End-PGH vs. non-PGH

In [22]:
cols = ['d_cast', 'd_reg']
precincts_pgh_ee = [
    'PITTSBURGH WARD 7',
    'PITTSBURGH WARD 8',
    'PITTSBURGH WARD 10',
    'PITTSBURGH WARD 11',
    'PITTSBURGH WARD 12',
    'PITTSBURGH WARD 13',
    'PITTSBURGH WARD 14',
    'PITTSBURGH WARD 15'
]
primaries_by_precinct_types = pd.DataFrame([['PGH_EE'], ['PGH_EE_NON'], ['PGH_NON']], columns=['precinct_type'])

for year in OFF_YEARS:
    for col in cols:
        col_year = col + '_' + year
        col_year_sum = primaries[col_year].sum()
        col_year_sum_pgh = primaries[primaries.precinct.str.startswith('PITTSBURGH')][col_year].sum()

        primaries_by_precinct_types.loc[
            primaries_by_precinct_types.precinct_type == 'PGH_EE',
            col_year
        ] = primaries[
            primaries.precinct.isin(precincts_pgh_ee)
        ][col_year].sum() / col_year_sum

        primaries_by_precinct_types.loc[
            primaries_by_precinct_types.precinct_type == 'PGH_EE',
            col + '_city_' + year
        ] = primaries[
            primaries.precinct.isin(precincts_pgh_ee)
        ][col_year].sum() / col_year_sum_pgh

        primaries_by_precinct_types.loc[
            primaries_by_precinct_types.precinct_type == 'PGH_EE_NON',
            col_year
        ] = primaries[
            (primaries.precinct.str.startswith('PITTSBURGH')) &
            ~(primaries.precinct.isin(precincts_pgh_ee))
        ][col_year].sum() / col_year_sum

        primaries_by_precinct_types.loc[
            primaries_by_precinct_types.precinct_type == 'PGH_EE_NON',
            col + '_city_' + year
        ] = primaries[
            (primaries.precinct.str.startswith('PITTSBURGH')) &
            ~(primaries.precinct.isin(precincts_pgh_ee))
        ][col_year].sum() / col_year_sum_pgh

        primaries_by_precinct_types.loc[
            primaries_by_precinct_types.precinct_type == 'PGH_NON',
            col_year
        ] = primaries[
            ~primaries.precinct.str.startswith('PITTSBURGH')
        ][col_year].sum() / col_year_sum

primaries_by_precinct_types.loc['Total'] = primaries_by_precinct_types.sum(numeric_only=True)

In [23]:
primaries_by_precinct_types

,precinct_type,d_cast_11,d_cast_city_11,d_reg_11,d_reg_city_11,d_cast_13,d_cast_city_13,d_reg_13,d_reg_city_13,d_cast_15,...,d_reg_15,d_reg_city_15,d_cast_17,d_cast_city_17,d_reg_17,d_reg_city_17,d_cast_19,d_cast_city_19,d_reg_19,d_reg_city_19
0,PGH_EE,0.129011,0.448155,0.125095,0.405878,0.164790,0.432273,0.126744,0.407845,0.133306,...,0.124734,0.407591,0.159547,0.476317,0.131162,0.412327,0.152042,0.479687,0.134201,0.417821
1,PGH_EE_NON,0.158861,0.551845,0.183113,0.594122,0.216427,0.567727,0.184021,0.592155,0.165150,...,0.181294,0.592409,0.175412,0.523683,0.186940,0.587673,0.164918,0.520313,0.186992,0.582179
2,PGH_NON,0.712127,NaN,0.691792,NaN,0.618783,NaN,0.689236,NaN,0.701544,...,0.693972,NaN,0.665042,NaN,0.681898,NaN,0.683040,NaN,0.678807,NaN
Total,NaN,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


##### Calculate PGH ward election turnout figures

In [24]:
primaries['d_turnout_avg'] = primaries[[
    c for c in primaries.columns if c.startswith('turnout') and c[-2:] in OFF_YEARS
]].mean(axis=1)

In [25]:
primaries[
    primaries.precinct.str.startswith('PITTSBURGH')
].sort_values(by='d_turnout_avg', ascending=False)[[
    'precinct', 'd_turnout_avg'
]].head(5)

,precinct,d_turnout_avg
104,PITTSBURGH WARD 24,0.318928
93,PITTSBURGH WARD 14,0.314038
89,PITTSBURGH WARD 10,0.285070
119,PITTSBURGH WARD 9,0.275451
107,PITTSBURGH WARD 27,0.271898


### Compare on- and off-year election

##### Calculate countywide election turnout figures

In [26]:
primaries_turnout = pd.DataFrame([['COUNTY']], columns=['precinct_type'])

for year in YEARS:
    primaries_turnout.loc[
        primaries_turnout.precinct_type == 'COUNTY',
        'd_turnout_' + year
    ] = primaries['d_cast_' + year].sum() / primaries['d_reg_' + year].sum()

primaries_turnout['d_turnout_offavg'] = primaries_turnout[[
    c for c in primaries_turnout.columns if c.startswith('d_turnout') and c[-2:] in OFF_YEARS
]].mean(axis=1)

primaries_turnout['d_turnout_onavg'] = primaries_turnout[[
    c for c in primaries_turnout.columns if c.startswith('d_turnout') and c[-2:] in ON_YEARS
]].mean(axis=1)

primaries_turnout

,precinct_type,d_turnout_10,d_turnout_11,d_turnout_12,d_turnout_13,d_turnout_14,d_turnout_15,d_turnout_16,d_turnout_17,d_turnout_18,d_turnout_19,d_turnout_offavg,d_turnout_onavg
0,COUNTY,0.284871,0.235999,0.199764,0.231976,0.204633,0.224402,0.449195,0.224951,0.227984,0.214001,0.226266,0.273289
